In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

## Original optimization problem

In [2]:
# name the variables, 12 is more understandable
var_names = ["environ_A","environ_B","environ_over", # grams of environment used and total overtime usage
             "Ag1_A","Ag2_A","Ag1_B","Ag2_B","Ag_over" # grams of A & B mix assigned to Agent 1 & 2, and total overtime work
            ]

In [3]:
# name the objective function
Profit = np.array([
    -11,-17.5,-4,82,87.5,82,87.5,-6.5
])

In [4]:
# name constraints matrix A
A = np.array([
    [1,1,0,0,0,0,0,0], # total of grams A&B <=1400
    [0,0,1,0,0,0,0,0], # total of grams A&B overtime <=400
    [1,1,-1,0,0,0,0,0], # regular usage of environment <=1000
    [0,0,0,1,1,1,1,0], # total work load of Agents <=1100
    [0,0,0,0,0,0,0,1], # total overtime work of Agents <=300
    [0,0,0,1,1,1,1,-1], # regular work of agents <=800
    [0,0,0,-0.1,0,-0.2,0.1,0], # % good in product >= 40%
    [0,0,0,-0.02,-0.01,0.03,-0.01,0], # % bad in product <= 7%
    [-0.44,0,0,1,1,0,0,0], # total work of Agent on mix A should not be larger than environment capacity
    [0,-0.72,0,0,0,1,1,0] # total work of Agent on mix B should not be larger than environment capacity
    ])

In [5]:
# name constraints vector b
rhs =np.array([1400,400,1000,1100,300,800,0,0,0,0])

In [6]:
# name linear optimization problem
m = gp.Model("critters")
x = m.addMVar(8,obj=Profit,name=var_names)
m.setObjective(Profit @ x, GRB.MAXIMIZE)
m.addConstr(A @ x <= rhs)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-18


<MConstr (10,) *awaiting model update*>

In [7]:
# solve optimization
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 8 columns and 29 nonzeros
Model fingerprint: 0xa418d1ee
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 6 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.3700000e+04   1.518000e+02   0.000000e+00      0s
       4    5.5656812e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.565681250e+04


In [8]:
print("\n\n Total Profit = {0:,.2f} ".format(m.ObjVal))



 Total Profit = 55,656.81 


In [9]:
m.printAttr(['X', 'Obj'])


    Variable            X          Obj 
--------------------------------------
   environ_A      196.875          -11 
   environ_B      1203.12        -17.5 
environ_over          400           -4 
       Ag1_A       86.625           82 
       Ag2_A            0         87.5 
       Ag1_B      259.875           82 
       Ag2_B      606.375         87.5 
     Ag_over      152.875         -6.5 


In [10]:
m.printAttr(['RHS', 'Sense', 'Slack'])


  Constraint          RHS        Sense        Slack 
---------------------------------------------------
          R0         1400            <            0 
          R1          400            <            0 
          R2         1000            <            0 
          R3         1100            <      147.125 
          R4          300            <      147.125 
          R5          800            <            0 
          R6           -0            <            0 
          R7           -0            <            0 
          R8           -0            <            0 
          R9           -0            <            0 


## Sensitivity analysis

In [11]:
def solve_print():
    m.reset()
    m.optimize()
    print("Total Profit = {0:.2f} ".format(m.ObjVal))
    m.printAttr(['X', 'Obj', 'rc', 'saobjlow', 'saobjup'])
    m.printAttr(['RHS', 'Sense', 'Slack', 'pi', 'sarhslow', 'sarhsup' ])

def set_obj (i, j):
    m.getVars()[i].obj = j

def set_rhs(i,j):
    m.getConstrs()[i].rhs=j

def new_obj_calc(i,j):
    save = m.getVars()[i].obj
    if j>0: new = objup[i] + epsilon
    else: new = objlow[i] - epsilon
    print("\n\n New Obj ",i," = ",new)
    set_obj(i,new)
    solve_print()
    set_obj(i,save)
    m.update()

def new_rhs_calc(i,j):
    save = m.getConstrs()[i].rhs
    if j>0: new = rhsup[i] + epsilon
    else: new = rhslow[i] - epsilon 
    print("\n\n New RHS ",i," = ",new)
    set_rhs(i,new)
    solve_print()
    set_rhs(i,save)
    m.update()

In [12]:
# name linear optimization problem
m = gp.Model("critters")
x = m.addMVar(8,obj=Profit,name=var_names)
m.setObjective(Profit @ x, GRB.MAXIMIZE)
m.addConstr(A @ x <= rhs)
m.update()

# Solve original problem
solve_print()

Discarded solution information
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 8 columns and 29 nonzeros
Model fingerprint: 0xa418d1ee
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 6 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.3700000e+04   1.518000e+02   0.000000e+00      0s
       4    5.5656812e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.565681250e+04
Total Profit = 55656.81 

    Variable            X          Obj           rc     saobjlow      saobjup 
-------------------------------------

In [13]:
# save the original problem details before sensitivity
objup = m.getAttr('SaObjUp')
objlow = m.getAttr('SaObjlow')
rhsup = m.getAttr('SaRhsUp')
rhslow = m.getAttr('SaRhsLow')

In [14]:
# # can try different variables
# epsilon = 0.1
# # Check the A sensitivity
# new_obj_calc(0,1)
# new_obj_calc(0,-1)

## e)

In [14]:
# will change the A matrix
obj=[]
A_new=A.copy()
for i in [0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10]:
    for j in [0,0.01,0.02,0.03,0.04,0.05]:
        
        A_new[8][0]=A[8][0]-0.44*i
        A_new[9][1]=A[9][1]-0.72*j
        
        m = gp.Model("critters")
        x = m.addMVar(8,obj=Profit,name=var_names)
        m.setObjective(Profit @ x, GRB.MAXIMIZE)
        m.addConstr(A_new @ x <= rhs)
        m.optimize()
        obj+=[m.ObjVal]
# obj

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 8 columns and 29 nonzeros
Model fingerprint: 0xa418d1ee
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 6 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.3700000e+04   1.518000e+02   0.000000e+00      0s
       4    5.5656812e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.565681250e+04
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, usin

In [15]:
obj_sublists = [obj[i:i+6] for i in range(0, len(obj), 6)]
tbl_e=pd.DataFrame(obj_sublists)
tbl_e.index=['A_yield+'+str(i)+'%' for i in range(0,11)]
tbl_e.columns=['B_yield+'+str(i)+'%' for i in range(0,6)]
display(tbl_e)

,B_yield+0%,B_yield+1%,B_yield+2%,B_yield+3%,B_yield+4%,B_yield+5%
A_yield+0%,55656.812500,56313.798721,56968.942350,57622.251128,58273.732753,58923.394880
A_yield+1%,55750.865376,56409.583750,57066.470372,57721.532873,58374.778838,59026.215815
A_yield+2%,55843.329032,56503.752526,57162.355000,57819.143974,58474.126929,59127.311304
A_yield+3%,55934.243412,56596.345616,57256.637424,57915.126250,58571.819470,59226.724418
A_yield+4%,56023.647130,56687.402239,57349.357469,58009.520132,58667.897500,59324.496804
A_yield+5%,56111.577528,56776.960323,57440.553653,58102.364727,58762.400715,59420.668750
A_yield+6%,56198.070728,56865.056560,57530.263237,58193.697866,58855.367520,59515.279232
A_yield+7%,56283.161680,56951.726453,57618.522275,58283.556160,58946.835080,59608.365974
A_yield+8%,56366.884211,57037.004363,57705.365669,58371.975046,59036.839372,59699.965490
A_yield+9%,56449.271066,57120.923557,57790.827207,58458.988836,59125.415230,59790.113141


In [16]:
# or, 0.44-0.54,0.72-0.77
obj=[]
A_new=A.copy()
for i in [0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10]:
    for j in [0,0.01,0.02,0.03,0.04,0.05]:
        
        A_new[8][0]=A[8][0]-i
        A_new[9][1]=A[9][1]-j
        
        m = gp.Model("critters")
        x = m.addMVar(8,obj=Profit,name=var_names)
        m.setObjective(Profit @ x, GRB.MAXIMIZE)
        m.addConstr(A_new @ x <= rhs)
        m.optimize()
        obj+=[m.ObjVal]
# obj

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 8 columns and 29 nonzeros
Model fingerprint: 0xa418d1ee
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.00s
Presolved: 6 rows, 6 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.3700000e+04   1.518000e+02   0.000000e+00      0s
       4    5.5656812e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.565681250e+04
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, usin

In [17]:
obj_sublists = [obj[i:i+6] for i in range(0, len(obj), 6)]
tbl_e2=pd.DataFrame(obj_sublists)
tbl_e2.index=['A_yield='+str(44+i)+'%' for i in range(0,11)]
tbl_e2.columns=['B_yield='+str(72+i)+'%' for i in range(0,6)]
display(tbl_e2)

,B_yield=72%,B_yield=73%,B_yield=74%,B_yield=75%,B_yield=76%,B_yield=77%
A_yield=44%,55656.812500,56568.795322,57477.229572,58382.135922,59283.534884,60181.446809
A_yield=45%,55868.275862,56785.678776,57699.580153,58610.000000,59516.958175,60420.474383
A_yield=46%,56071.789474,56994.424015,57913.602996,58829.345794,59741.671642,60650.599628
A_yield=47%,56267.793358,57195.480663,58119.757353,59040.642202,59958.153846,60872.310786
A_yield=48%,56456.695652,57389.265823,58318.469314,59244.324324,60166.848921,61086.061041
A_yield=49%,56638.875445,57576.166963,58510.134752,59440.796460,60368.169611,61292.271605
A_yield=50%,56814.685315,57756.544503,58695.121951,59630.434783,60562.500000,61491.334489
A_yield=51%,56984.453608,57930.734134,58873.773973,59813.589744,60750.197952,61683.614991
A_yield=52%,57148.486486,58099.048904,59046.410774,59990.588235,60931.597315,61869.453936
A_yield=53%,57307.069767,58261.781095,59213.331126,60161.735537,61107.009901,62049.169687


## f)

In [18]:
# will change the A matrix
obj=[]
A_new=A.copy()
for i in [0.40,0.39,0.38,0.37,0.36,0.35]:
    for j in [0.07,0.08,0.09,0.10]:
        A_new[6]=np.array([0,0,0,i-0.5,i-0.4,i-0.6,i-0.3,0])
        A_new[7]=np.array([0,0,0,0.05-j,0.06-j,0.1-j,0.06-j,0])
        m = gp.Model("critters")
        x = m.addMVar(8,obj=Profit,name=var_names)
        m.setObjective(Profit @ x, GRB.MAXIMIZE)
        m.addConstr(A_new @ x <= rhs)
        m.optimize()
        obj+=[m.ObjVal]
# obj

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 8 columns and 29 nonzeros
Model fingerprint: 0x88a504bb
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 6 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.3700000e+04   1.518000e+02   0.000000e+00      0s
       4    5.5656812e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.565681250e+04
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, usin

In [19]:
obj_sublists = [obj[i:i+4] for i in range(0, len(obj), 4)]
tbl_f=pd.DataFrame(obj_sublists)
tbl_f.index=['limit_good='+str(i)+'%' for i in [40,39,38,37,36,35]]
tbl_f.columns=['limit_bad='+str(i)+'%' for i in [7,8,9,10]]
display(tbl_f)

,limit_bad=7%,limit_bad=8%,limit_bad=9%,limit_bad=10%
limit_good=40%,55656.812500,58900.0,58900.0,58900.0
limit_good=39%,57089.169329,59084.8,59084.8,59084.8
limit_good=38%,58587.058824,59269.6,59269.6,59269.6
limit_good=37%,59454.400000,59454.4,59454.4,59454.4
limit_good=36%,59639.200000,59639.2,59639.2,59639.2
limit_good=35%,59824.000000,59824.0,59824.0,59824.0


## g)

In [20]:
# similar to add an more expensive option of overtime working
var_names_new=var_names+['A_newstuff','B_newstuff','A_Ag1_newstuff','A_Ag2_newstuff','B_Ag1_newstuff','B_Ag2_newstuff']

In [21]:
# name the objective function
Profit_new = np.array([
    -11,-17.5,-4,82,87.5,82,87.5,-6.5,-30,-30,80,80,80,80
])

In [22]:
# name constraints matrix A
A_new = np.array([
    [1,1,0,0,0,0,0,0,0,0,0,0,0,0], # total of grams A&B <=1400
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0], # total of grams A&B overtime <=400
    [1,1,-1,0,0,0,0,0,0,0,0,0,0,0], # regular usage of environment <=1000
    [0,0,0,1,1,1,1,0,0,0,0,0,0,0], # total work load of Agents <=1100
    [0,0,0,0,0,0,0,1,0,0,0,0,0,0], # total overtime work of Agents <=300
    [0,0,0,1,1,1,1,-1,0,0,0,0,0,0], # regular work of agents <=800
    [0,0,0,-0.1,0,-0.2,0.1,0,0,0,-0.1,0,-0.2,0.1], # % good in product >= 40%
    [0,0,0,-0.02,-0.01,0.03,-0.01,0,0,0,-0.02,-0.01,0.03,-0.01], # % bad in product <= 7%
    [-0.44,0,0,1,1,0,0,0,-0.44,0,1,1,0,0], # total work of Agent on mix A should not be larger than environment capacity
    [0,-0.72,0,0,0,1,1,0,0,-0.72,0,0,1,1] # total work of Agent on mix B should not be larger than environment capacity
    ])

In [23]:
# name linear optimization problem
m = gp.Model("critters")
x = m.addMVar(14,obj=Profit_new,name=var_names_new)
m.setObjective(Profit_new @ x, GRB.MAXIMIZE)
m.addConstr(A_new @ x <= rhs)

# solve optimization
m.optimize()

# m.printAttr(['X', 'Obj'])

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 14 columns and 42 nonzeros
Model fingerprint: 0x3420aa11
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.02s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Infeasible or unbounded model


In [24]:
# it turns the model is unbounded, so we foucus on the best distribution of new hires in departments
# since problem is linear, we can assume that total new hire = 1,100,1000,10000,...
# we can use a <= constraint since == will definitely yield the highest profit

In [25]:
# name constraints matrix A
A_new = np.array([
    [1,1,0,0,0,0,0,0,0,0,0,0,0,0], # total of grams A&B <=1400
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0], # total of grams A&B overtime <=400
    [1,1,-1,0,0,0,0,0,0,0,0,0,0,0], # regular usage of environment <=1000
    [0,0,0,1,1,1,1,0,0,0,0,0,0,0], # total work load of Agents <=1100
    [0,0,0,0,0,0,0,1,0,0,0,0,0,0], # total overtime work of Agents <=300
    [0,0,0,1,1,1,1,-1,0,0,0,0,0,0], # regular work of agents <=800
    [0,0,0,-0.1,0,-0.2,0.1,0,0,0,-0.1,0,-0.2,0.1], # % good in product >= 40%
    [0,0,0,-0.02,-0.01,0.03,-0.01,0,0,0,-0.02,-0.01,0.03,-0.01], # % bad in product <= 7%
    [-0.44,0,0,1,1,0,0,0,-0.44,0,1,1,0,0], # total work of Agent on mix A should not be larger than environment capacity
    [0,-0.72,0,0,0,1,1,0,0,-0.72,0,0,1,1], # total work of Agent on mix B should not be larger than environment capacity
    [0,0,0,0,0,0,0,0,1,1,1,1,1,1]
    ])

# name constraints vector b
rhs_new =np.array([1400,400,1000,1100,300,800,0,0,0,0,1000])

In [26]:
m = gp.Model("critters")
x = m.addMVar(14,obj=Profit_new,name=var_names_new)
m.setObjective(Profit_new @ x, GRB.MAXIMIZE)
m.addConstr(A_new @ x <= rhs_new)

# solve optimization
m.optimize()

# m.printAttr(['X', 'Obj'])

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-8559U CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 14 columns and 48 nonzeros
Model fingerprint: 0x10ecce05
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 9 rows, 14 columns, 46 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7625000e+05   3.000000e+02   0.000000e+00      0s
       9    7.3933191e+04   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.393319078e+04


In [27]:
m.printAttr(['X', 'Obj'])


    Variable            X          Obj 
--------------------------------------
   environ_A       292.86          -11 
   environ_B      1107.14        -17.5 
environ_over          400           -4 
       Ag1_A      128.858           82 
       Ag2_A            0         87.5 
       Ag1_B      69.1335           82 
       Ag2_B      902.008         87.5 
     Ag_over          300         -6.5 
  A_newstuff            0          -30 
  B_newstuff      682.559          -30 
A_Ag1_newstuff            0           80 
A_Ag2_newstuff            0           80 
B_Ag1_newstuff      317.441           80 
B_Ag2_newstuff            0           80 


In [31]:
# should hire more staff to harvest B, until more staff are needed to bake the mix
# then further hire more staff to harvest B and more Agent1 to bake mix B, keep the production balanced